In [57]:
import requests
from bs4 import BeautifulSoup
import json
import types
from os import listdir
from os.path import isfile, join
from unidecode import unidecode

LYRICSSITE = 'http://www.lyrics.wikia.com/api.php'
CHARTS_PATH = 'billboard100/'
LYRICS_PATH = 'lyrics/'
NUMBER, ARTIST, SONG = range(3)

In [71]:
def replace_with_newlines(elem,lyrics_soup):
    lyric = ''
    for br in lyrics_soup.findAll('br'):
        next_s = br.nextSibling
        next2_s = next_s.nextSibling
        text = str(next_s).strip()
        if "<" in text:
            pass
        else:
            lyric = " ".join([lyric,text])
    return lyric

In [72]:
for file in files:
    f = open(CHARTS_PATH+file)
    #if isfile(LYRICS_PATH+file):
    #    f.close()
    #    continue
    songs = f.readlines()
    f.close()
    year = file.strip('hot100.csv')
    lyrics_file = open(LYRICS_PATH+file, 'w') # truncates existing file
    
    for song in songs:
        song_params = song.split(',')

        params = {
            'action': 'lyrics',
            'artist': song_params[ARTIST],
            'song': song_params[SONG]
        }

        link_r = requests.get(LYRICSSITE, params = params)

        link_soup = BeautifulSoup(link_r.text, 'html.parser')
        link = 0
        anc = link_soup.a

        if link_soup.a:
            if link_soup.a.has_attr('href'):
                link = anc['href']
        if link != 0:
            lyrics_r = requests.get(link)

            lyrics_soup = BeautifulSoup(lyrics_r.text, 'html.parser')

            comments = lyrics_soup.findAll(text = lambda text: isinstance(text, Comment))
            # extract comments
            [c.extract() for c in comments]

            # extract <script> tags
            [s.extract() for s in lyrics_soup('script')]

            lyrics_divs = lyrics_soup.findAll('div', { 'class': 'lyricbox'})

            line = song.rstrip('\n') + ','
            line = ",".join([year,line])
            for div in lyrics_divs:
                lyric = replace_with_newlines(div,lyrics_soup)
                line += lyric
            line += '\n'
            lyrics_file.write(line)    
    lyrics_file.close()


'''
params = {
        'action': 'lyrics',
        'artist': 'j cole',
        'song': 'no role modelz'
        }

'''
#Resource: https://github.com/alexmwu/hot100

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x87 in position 23: invalid start byte